# Bank Customer Exit Risk Analysis


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/H/Downloads/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.0.0'

In [4]:
#Taking columns from credit score as number,id,name doesn't help
x = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [5]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


In [7]:
#Label encode Gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [8]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [10]:
#One hot encode geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [12]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .2, random_state=0)


In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [17]:
an = tf.keras.models.Sequential()
an.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [18]:
an.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [19]:
#activation = softmax if not binary classification
an.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [20]:
an.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [40]:
#Batch Learning
an.fit(x_train, y_train, batch_size = 64, epochs = 100,)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 0s 47us/sample - loss: 0.3288 - accuracy: 0.8646
Epoch 2/100
8000/8000 [==============================] - 0s 16us/sample - loss: 0.3287 - accuracy: 0.8649
Epoch 3/100
8000/8000 [==============================] - 0s 16us/sample - loss: 0.3288 - accuracy: 0.8649
Epoch 4/100
8000/8000 [==============================] - 0s 16us/sample - loss: 0.3287 - accuracy: 0.8655
Epoch 5/100
8000/8000 [==============================] - 0s 16us/sample - loss: 0.3287 - accuracy: 0.8651
Epoch 6/100
8000/8000 [==============================] - 0s 17us/sample - loss: 0.3284 - accuracy: 0.8651
Epoch 7/100
8000/8000 [==============================] - 0s 16us/sample - loss: 0.3289 - accuracy: 0.8641
Epoch 8/100
8000/8000 [==============================] - 0s 17us/sample - loss: 0.3287 - accuracy: 0.8662
Epoch 9/100
8000/8000 [==============================] - 0s 17us/sample - loss: 0.3288 - accuracy: 0.8660
Epoch 10/100
8000/8000 [

In [41]:
print(an.predict(sc.transform([[ 1, 0, 0, 500, 0, 55, 2, 100000, 2, 1, 1, 221200]])))#.5 is threshold probability and if it is greater than .5 then the probability of leaving is high

[[0.20375568]]


In [42]:
y_pred = an.predict(x_test)
y_pred = (y_pred > .5)

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1514   81]
 [ 193  212]]


0.863